In [3]:
%time for i in range(100000000): pass

CPU times: user 4.35 s, sys: 0 ns, total: 4.35 s
Wall time: 4.35 s


In [ ]:
from IPython.display import YouTubeVideo,FileLink,FileLinks

In [ ]:
YouTubeVideo("no2dG9OlXZo")

In [ ]:
FileLinks('./')

In [1]:
import numpy as np

In [ ]:
x = y = z = np.arange(0.0,5.0,1.0)
np.savetxt('test1.csv', x, delimiter=',')   # X is an array
np.savetxt('test2.out', (x,y,z))   # x,y,z equal sized 1D arrays
np.savetxt('test3.out', x, fmt='%1.4e')   # use exponential notation

In [ ]:
x = np.array([[1,2],[3,4]])
np.savetxt('test1.csv', x, delimiter=',')

In [2]:
import pandas as pd
import numpy as np
data = np.array(['a','b','c','d'])
s = pd.Series(data)
print(s)

0    a
1    b
2    c
3    d
dtype: object


In [ ]:
type(s)

In [3]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()
np.array( ratings.userId.unique() )

NameError: name 'path' is not defined

In [ ]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title']
print(movie_names.index)

In [ ]:
a = pd.read_csv(path+'movies.csv').title

In [ ]:
print(a)

In [22]:
import tensorflow as tf

In [5]:
import pandas as pd
import numpy as np

In [6]:
path = "CS231n/assignment2/ml-latest-small/"

In [7]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [9]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [10]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [11]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [12]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [13]:
n_factors = 50

In [14]:
np.random.seed = 42

In [15]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [16]:
print( ratings.shape )
print( trn.shape )
print( val.shape )

(100004, 4)
(79721, 4)
(20283, 4)


In [17]:
a = np.array([True,False])
print(a)
print(~a)

[ True False]
[False  True]


In [18]:
print( trn.head() )
print()
print( trn.userId.head() )
print()
print( trn.movieId.head() )

   userId  movieId  rating   timestamp
0       0        0     2.5  1260759144
1       0        1     3.0  1260759179
2       0        2     3.0  1260759182
4       0        4     4.0  1260759205
5       0        5     2.0  1260759151

0    0
1    0
2    0
4    0
5    0
Name: userId, dtype: int64

0    0
1    1
2    2
4    4
5    5
Name: movieId, dtype: int64


In [19]:
inp = np.array( trn.movieId )
print( inp[:3])

[0 1 2]


In [20]:
import math

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(1)

user_id = tf.placeholder(tf.int32, shape=[None] )
movie_id = tf.placeholder(tf.int32, shape=[None] )
rating =tf.placeholder(tf.float32, shape=[None] )

reg = tf.contrib.layers.l2_regularizer(scale=1e-4)

user_em = tf.get_variable("user_em",shape = [n_users,n_factors] , regularizer=reg )
movie_em = tf.get_variable("movie_em",shape = [n_movies,n_factors], regularizer=reg )

user_bi = tf.get_variable("bi1",shape=[n_users],initializer=tf.zeros_initializer )
movie_bi = tf.get_variable("bi2",shape=[n_movies],initializer=tf.zeros_initializer )

x = tf.nn.embedding_lookup( params=user_em , ids= user_id )
y = tf.nn.embedding_lookup( params=movie_em , ids= movie_id )

xx = tf.nn.embedding_lookup( params=user_bi , ids= user_id )
yy = tf.nn.embedding_lookup( params=movie_bi , ids= movie_id )

z = tf.reduce_sum( x*y , axis = 1 ) + xx + yy

mean_loss = tf.losses.mean_squared_error(z,rating)
final_loss = mean_loss + tf.losses.get_regularization_loss()

train_op = tf.train.AdamOptimizer(0.001).minimize(final_loss)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    
    ui = np.array( trn.userId )
    mi = np.array( trn.movieId )
    rt = np.array( trn.rating )

    batch_size = 64
    
    for e in range(10):
        
        train_indicies = np.arange(ui.shape[0])
        np.random.shuffle(train_indicies)
        for i in range(int(math.ceil(ui.shape[0]/batch_size))):
            
            start_idx = (i*batch_size)%ui.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            feed_dict={user_id: ui[idx] , movie_id: mi[idx] ,rating: rt[idx] } 
       
            _,loss = sess.run([ train_op , mean_loss ] ,feed_dict=feed_dict)
        
        print( sess.run( final_loss , feed_dict={user_id: ui , movie_id: mi ,rating: rt } ) )

In [30]:
tf.reset_default_graph()
tf.set_random_seed(1)

user_id = tf.placeholder(tf.int32, shape=[None] )
movie_id = tf.placeholder(tf.int32, shape=[None] )
rating =tf.placeholder(tf.float32, shape=[None] )


user_em = tf.get_variable("user_em",shape = [n_users,n_factors]  )
movie_em = tf.get_variable("movie_em",shape = [n_movies,n_factors] )

user_bi = tf.get_variable("bi1",shape=[n_users],initializer=tf.zeros_initializer )
movie_bi = tf.get_variable("bi2",shape=[n_movies],initializer=tf.zeros_initializer )

x = tf.nn.embedding_lookup( params=user_em , ids= user_id )
y = tf.nn.embedding_lookup( params=movie_em , ids= movie_id )

z = tf.concat( [x,y] , axis=1 )
z = tf.layers.dropout( z,0.2 )

z = tf.layers.dense( z , 70 , activation=tf.nn.relu )
z = tf.layers.dropout( z,0.75 )

z = tf.layers.dense( z , 1 )
z = tf.reshape(z , [-1])

final_loss = tf.losses.mean_squared_error(z,rating)

train_op = tf.train.AdamOptimizer(0.001).minimize(final_loss)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    
    ui = np.array( trn.userId )
    mi = np.array( trn.movieId )
    rt = np.array( trn.rating )

#     feed_dict={user_id: ui , movie_id: mi ,rating: rt }   
    batch_size = 64
#     q,w = sess.run( [z,rating] , feed_dict=feed_dict)
#     print( q.shape , w.shape )
    
    for e in range(10):
        
        train_indicies = np.arange(ui.shape[0])
        np.random.shuffle(train_indicies)
        for i in range(int(math.ceil(ui.shape[0]/batch_size))):
            
            start_idx = (i*batch_size)%ui.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            feed_dict={user_id: ui[idx] , movie_id: mi[idx] ,rating: rt[idx] } 
       
            _ = sess.run([ train_op ] ,feed_dict=feed_dict)
        
        print( sess.run( final_loss , feed_dict={user_id: ui , movie_id: mi ,rating: rt } ) )

0.744558
0.674704
0.643131
0.615137
0.597676
0.571575
0.529277
0.499136
0.472052
0.430214
